## ML-7. Прогнозирование биологического ответа (HW-3)

### 4. Практика
#### Практика основана на соревновании Kaggle: Predicting a Biological Response (Прогнозирование биологического ответа).
#### Необходимо предсказать биологический ответ молекул (столбец 'Activity') по их химическому составу (столбцы D1-D1776).
#### Каждая строка представляет молекулу. 

- Первый столбец Activity содержит экспериментальные данные, описывающие фактический биологический ответ [0, 1]; 

- Остальные столбцы D1-D1776 представляют собой молекулярные дескрипторы — это вычисляемые свойства, которые могут фиксировать некоторые характеристики молекулы, например размер, форму или состав элементов.

#### Предварительная обработка не требуется, данные уже закодированы и нормализованы.
#### В качестве метрики будем использовать F1-score.
#### Необходимо обучить две модели: **логистическую регрессию** и **случайный лес**. 
#### Далее нужно сделать *подбор гиперпараметров* с помощью базовых и продвинутых методов оптимизации. Важно использовать все четыре метода (**GridSeachCV**, **RandomizedSearchCV**, **Hyperopt**, **Optuna**) хотя бы по одному разу. 
#### **Максимальное количество итераций не должно превышать 50**.

In [1]:
#импорт библиотек
import numpy as np #для матричных вычислений
import pandas as pd #для анализа и предобработки данных
import matplotlib.pyplot as plt #для визуализации
import seaborn as sns #для визуализации

from sklearn import linear_model #линейные модели
from sklearn import tree #деревья решений
from sklearn import ensemble #ансамбли
from sklearn import metrics #метрики
from sklearn import preprocessing #предобработка
from sklearn.model_selection import train_test_split #сплитование выборки

%matplotlib inline
plt.style.use('seaborn')

In [2]:
# подключаем google диск, где хранятся данные
from google.colab import drive
drive.mount('/content/drive')

# берем необходимый файл с диска, указав путь
data = pd.read_csv('./drive/MyDrive/SF/train_molecular.csv')
data.head()

Mounted at /content/drive


,Activity,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D1767,D1768,D1769,D1770,D1771,D1772,D1773,D1774,D1775,D1776
0,1,0.000000,0.497009,0.10,0.0,0.132956,0.678031,0.273166,0.585445,0.743663,...,0,0,0,0,0,0,0,0,0,0
1,1,0.366667,0.606291,0.05,0.0,0.111209,0.803455,0.106105,0.411754,0.836582,...,1,1,1,1,0,1,0,0,1,0
2,1,0.033300,0.480124,0.00,0.0,0.209791,0.610350,0.356453,0.517720,0.679051,...,0,0,0,0,0,0,0,0,0,0
3,1,0.000000,0.538825,0.00,0.5,0.196344,0.724230,0.235606,0.288764,0.805110,...,0,0,0,0,0,0,0,0,0,0
4,0,0.100000,0.517794,0.00,0.0,0.494734,0.781422,0.154361,0.303809,0.812646,...,0,0,0,0,0,0,0,0,0,0


## 1. Логистическая регрессия
#### Создаем матрицу наблюдений **X** и вектор ответов **y**

In [3]:
X = data.drop(['Activity'], axis=1)
y = data['Activity']

#### Разделяем выборку на *тренировочную* и *тестовую* в соотношении **80/20**. Для сохранения соотношений целевого признака используем параметр **stratify** (стратифицированное разбиение).

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state = 1, test_size = 0.2)

#### Зафиксируем метрики со значениями гиперпараметров, установленных по умолчанию:

In [5]:
#Создаем объект класса логистическая регрессия
log_reg = linear_model.LogisticRegression(max_iter = 1000)
#Обучаем модель, минимизируя logloss
log_reg.fit(X_train, y_train)

y_test_pred = log_reg.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.78


## 1.1 GridSearchCV

In [6]:
from sklearn.model_selection import GridSearchCV

param_grid = {'penalty': ['l2', 'none'] ,#тип регурялизации
              'solver': ['lbfgs', 'saga'], #алгоритм оптимизации
              }
grid_search = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=42, #генератор случайных чисел
        max_iter=50 #количество итераций на сходимость
    ), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search.fit(X_train, y_train) 
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

CPU times: user 3.83 s, sys: 86.6 ms, total: 3.91 s
Wall time: 32.2 s
f1_score на тестовом наборе: 0.78
Наилучшие значения гиперпараметров: {'penalty': 'l2', 'solver': 'saga'}


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


#### Значение метрики не изменилось, это значит, что мы не нашли комбинацию внешних параметров лучше, чем заданы по умолчанию.
#### Попробуем расширить сетку гиперпараметров:

In [7]:
param_grid = [
              {'penalty': ['l2', 'none'] , # тип регуляризации
              'solver': ['lbfgs', 'sag'], # алгоритм оптимизации
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}, # уровень силы регурялизации
              
              {'penalty': ['l1', 'l2'] ,
              'solver': ['liblinear', 'saga'],
               'C': [0.01, 0.1, 0.3, 0.5, 0.7, 0.9, 1]}
]
grid_search_1 = GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=1, max_iter=50), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search_1.fit(X_train, y_train) 
y_test_pred = grid_search_1.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_1.best_params_))

CPU times: user 3.92 s, sys: 348 ms, total: 4.27 s
Wall time: 6min 21s
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'C': 0.1, 'penalty': 'l2', 'solver': 'lbfgs'}


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


#### Метрика незначительно улучшилась, но времени ушло гораздо больше.
#### Ознакомимся с итоговой полученной моделью с помощью best_estimator_: 

In [8]:
print("Наилучшая модель:\n{}".format(grid_search.best_estimator_))

Наилучшая модель:
LogisticRegression(max_iter=50, random_state=42, solver='saga')


#### Посмотрим на наилучшее значение точности кросс-валидации (значение точности, усредненное по всем разбиениям для данной комбинации гиперпараметров) – в атрибуте best_score_: 

In [9]:
print("Наилучшее значение точности при кросс-валидации: {:.2f}".format(grid_search.best_score_))

Наилучшее значение точности при кросс-валидации: 0.76


## 1.2 RandomizedSearchCV

In [10]:
from sklearn.model_selection import RandomizedSearchCV

#np.linspace(start(от), stop(до), num=50(количество),dtype- тип данных)
param_grid = {'penalty': ['l2', 'none'] ,
              'solver': ['lbfgs', 'sag'],
               'C': list(np.linspace(0.01, 1, 10, dtype=float))},
            
random_search = RandomizedSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=50), 
    param_distributions=param_grid, 
    cv=5, 
    n_iter = 10, 
    n_jobs = -1
)  
%time random_search.fit(X_train, y_train) 
y_test_pred = random_search.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(random_search.best_params_))

CPU times: user 3.52 s, sys: 120 ms, total: 3.64 s
Wall time: 1min 18s
f1_score на тестовом наборе: 0.78
Наилучшие значения гиперпараметров: {'solver': 'sag', 'penalty': 'l2', 'C': 0.23}


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


#### За 10 итераций метрику улучшить не удалось, но время уменьшилось в 6 раз. Зададим аналогичную сетку и проделаем тоже с помощью GridSearchCV и сравним:

In [11]:
param_grid = {'penalty': ['l2', 'none'] ,
              'solver': ['lbfgs', 'sag'],
               'C': list(np.linspace(0.01, 1, 10, dtype=float))},
            
grid_search_2 = GridSearchCV(
    estimator=linear_model.LogisticRegression(random_state=42, max_iter=50), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search_2.fit(X_train, y_train) 
y_test_pred = grid_search_2.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search_2.best_params_))

CPU times: user 5 s, sys: 260 ms, total: 5.26 s
Wall time: 3min 47s
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'C': 0.12, 'penalty': 'l2', 'solver': 'sag'}


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


#### Время увеличилось более, чем в 3 раза! Метрики не изменились
## 1.3 Hyperopt
#### Продвинутая оптимизация не получилась, так как не было образца

In [1]:
#%pip install hyperopt

     |████████████████████████████████| 1.6 MB 816 kB/s eta 0:00:01     |█████████████████               | 839 kB 816 kB/s eta 0:00:01
     |████████████████████████████████| 199 kB 16.5 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
#делаем импорт и выведем версию библиотеки
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
# fmin - основная функция, она будет минимизировать наш функционал
# tpe - алгоритм оптимизации
# hp - включает набор методов для объявления пространства поиска гиперпараметров
# trails - используется для логирования результатов

print("Версия Hyperopt : {}".format(hyperopt.__version__))

Версия Hyperopt : 0.2.7


In [13]:
# зададим пространство поиска гиперпараметров
space = {
        'lr__penalty' : hp.choice(label='penalty', 
                        options=['l1', 'l2']),
        'lr__C' : hp.loguniform(label='C', 
                        low=-4*np.log(10), 
                        high=2*np.log(10))
        }

#### Здесь параметр регуляризации C выбирается из лог-равномерного распределения [- 4ln10, 2ln10], и может принимать значения [10^-4, 10^2], а тип регуляризации равновероятно выбирается из [l1, l2]. Можно выбрать и другие типы распределений, например, равномерное, или нормальное.
#### Зададим функцию, которую будем оптимизировать. Она принимает на вход гиперпараметры, модель и данные, после чего возвращает точность на кросс-валидации:


In [14]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from hyperopt import hp, fmin, tpe, Trials, STATUS_OK
from sklearn.model_selection import KFold

def objective(params, pipeline,  X_train, y_train):
    """
    Кросс-валидация с текущими гиперпараметрами

    :params: гиперпараметры
    :pipeline: модель
    :X_train: матрица признаков
    :y_train: вектор меток объектов
    :return: средняя точность на кросс-валидации
    """ 

    # задаём модели требуемые параметры    
    pipeline.set_params(**params)

    # задаём параметры кросс-валидации (стратифицированная 4-фолдовая с перемешиванием)
    skf = StratifiedKFold(n_splits=4, shuffle=True, random_state=1)

    # проводим кросс-валидацию  
    score = cross_val_score(estimator=pipeline, X=X_train, y=y_train, 
                            scoring='roc_auc', cv=skf, n_jobs=-1)
    # возвращаем результаты, которые записываются в Trials()
    return   {'loss': -score.mean(), 'params': params, 'status': STATUS_OK}

In [15]:
grid_search_hp = GridSearchCV(
    estimator=linear_model.LogisticRegression(
        random_state=42, #генератор случайных чисел
        max_iter=50 #количество итераций на сходимость
    ), 
    param_grid=param_grid, 
    cv=5, 
    n_jobs = -1
)  
%time grid_search.fit(X_train, y_train) 
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))
print("Наилучшие значения гиперпараметров: {}".format(grid_search.best_params_))

CPU times: user 3.79 s, sys: 42.4 ms, total: 3.83 s
Wall time: 31 s
f1_score на тестовом наборе: 0.79
Наилучшие значения гиперпараметров: {'penalty': 'l2', 'solver': 'saga'}


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,


## 2. Случайный лес
## 2.1 Базовая оптимизация
#### В sklearn все ансамблевые методы реализованы в модуле **ensemble**.
#### В sklearn модель случайного леса для решения задачи классификации реализована в классе **RandomForestClassifier**.



In [24]:
#Создаем объект класса RandomForestClassifier (Случайный лес) 
rf_clf_full = ensemble.RandomForestClassifier(
    n_estimators=500, #число деревьев
    criterion='gini', #критерий эффективности
    max_depth=8, #максимальная глубина дерева
    max_features='sqrt', #число признаков из метода случайных подпространств
    min_samples_leaf=10, #минимальное число объектов в листе
    random_state=42 #генератор случайных чисел
)
#Обучаем модель 
rf_clf_full.fit(X_train, y_train)
#Делаем предсказание класса
y_test_pred = rf_clf_full.predict(X_test)
#Выводим отчёт о метрике f1-score
print(f'{metrics.f1_score(y_test, y_test_pred):.2f}')

0.80


#### Наша метрика немного f1-score улучшилась
## 2.2 Продвинутая оптимизация (Hyperopt)

In [25]:
#делаем импорт и выведем версию библиотеки
from sklearn.model_selection import cross_val_score
import hyperopt
from hyperopt import hp, fmin, tpe, Trials
# fmin - основная функция, она будет минимизировать наш функционал
# tpe - алгоритм оптимизации
# hp - включает набор методов для объявления пространства поиска гиперпараметров
# trails - используется для логирования результатов

print("Версия Hyperopt : {}".format(hyperopt.__version__))

Версия Hyperopt : 0.2


#### Настроим оптимизацию гиперпараметров для алгоритма случайного леса.

In [26]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 26, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }

#### Интерфейс **Hyperopt** отличается от Grid или RandomizedSearch, поэтому нам нужно создать *функцию для минимизации*. Она должна принимать словарь значений гиперпараметров и возвращать значение целевой функции.

In [27]:
# зафксируем random_state
random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

    # обучаем модель
    model.fit(X, y)
    score = metrics.f1_score(y, model.predict(X))
    
    # обучать модель можно также с помощью кросс-валидации
    # применим  cross validation с тем же количеством фолдов
    # score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    # метрику необходимо минимизировать, поэтому ставим знак минус
    return -score

#### Начинаем подбор гиперпараметров

In [28]:
%%time

trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.RandomState(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 20/20 [00:41<00:00,  2.07s/it, best loss: -0.9911233547597184]
Наилучшие значения гиперпараметров {'max_depth': 24.0, 'min_samples_leaf': 2.0, 'n_estimators': 153.0}
CPU times: user 40.9 s, sys: 80.1 ms, total: 41 s
Wall time: 41.4 s


In [29]:
# рассчитаем f1-score для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))


f1_score на тестовом наборе: 0.82


#### Метрика улучшилась и время уменьшилось.
#### Продолжим поиск гиперпараметров, чтобы получить лучшие результаты.
#### Для байесовских оптимизаторов это возможно, так как они ориентируются на прошлые результаты: предыдущие входные данные для целевой функции и результирующие потери.
#### Hyperopt продолжит поиск с того места, где он остановился, если мы передадим ему объект Trials, который уже содержит информацию о предыдущих запусках.
#### Увеличиваем max_evals до 50 (должен быть строго больше, чем на предыдущих итерациях) 
#### и используем старый trials

In [30]:
best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=50, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.RandomState(random_state)# фиксируем для повторяемости результата
         )
print("Наилучшие значения гиперпараметров {}".format(best))

100%|██████████| 30/30 [00:51<00:00,  1.72s/it, best loss: -0.9917304747320062]
Наилучшие значения гиперпараметров {'max_depth': 22.0, 'min_samples_leaf': 2.0, 'n_estimators': 114.0}


In [31]:
# Вновь рассчитаем f1-score для тестовой выборки
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.82


#### Метрика не изменилась.
## 2.3 Продвинутая оптимизация (Optuna)

In [33]:
# Устанавливаем библиотеку
#!pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 7.5 MB/s 
     |████████████████████████████████| 81 kB 11.8 MB/s 
     |████████████████████████████████| 209 kB 57.0 MB/s 
     |████████████████████████████████| 78 kB 8.7 MB/s 
     |████████████████████████████████| 112 kB 57.5 MB/s 
     |████████████████████████████████| 146 kB 58.1 MB/s 
     |████████████████████████████████| 49 kB 7.7 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=986c6d94da4152706051c79be0326f3c2e1e2e7d4e46708671552eecd4958383
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [34]:
import optuna

print("Версия Optuna: {}".format(optuna.__version__))

Версия Optuna: 2.10.1


#### Настроим оптимизацию гиперпараметров для алгоритма случайного леса:

In [35]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 10, 30, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  score = metrics.f1_score(y_train, model.predict(X_train))

  return score

In [36]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров 20 раз
study.optimize(optuna_rf, n_trials=20)

[I 2022-06-29 22:29:09,873] A new study created in memory with name: RandomForestClassifier
[I 2022-06-29 22:29:11,284] Trial 0 finished with value: 0.9274563820018367 and parameters: {'n_estimators': 113, 'max_depth': 13, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.9274563820018367.
[I 2022-06-29 22:29:13,520] Trial 1 finished with value: 0.9283318084781945 and parameters: {'n_estimators': 178, 'max_depth': 18, 'min_samples_leaf': 7}. Best is trial 1 with value: 0.9283318084781945.
[I 2022-06-29 22:29:15,507] Trial 2 finished with value: 0.9380530973451326 and parameters: {'n_estimators': 161, 'max_depth': 11, 'min_samples_leaf': 3}. Best is trial 2 with value: 0.9380530973451326.
[I 2022-06-29 22:29:17,791] Trial 3 finished with value: 0.906536346976176 and parameters: {'n_estimators': 198, 'max_depth': 30, 'min_samples_leaf': 9}. Best is trial 2 with value: 0.9380530973451326.
[I 2022-06-29 22:29:19,250] Trial 4 finished with value: 0.917302410741532 and parameters: {'n_es

CPU times: user 39.5 s, sys: 107 ms, total: 39.6 s
Wall time: 39.5 s


In [37]:
# рассчитаем f1_score для тестовой выборки
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.82


#### Метрика немного улучшилась
#### Продолжим подбор, указав n_trials=30, которое добавится к предыдущим 20 итерациям

In [38]:
%%time
study.optimize(optuna_rf, n_trials=30)

[I 2022-06-29 22:37:01,905] Trial 20 finished with value: 0.9761613691931541 and parameters: {'n_estimators': 130, 'max_depth': 21, 'min_samples_leaf': 3}. Best is trial 20 with value: 0.9761613691931541.
[I 2022-06-29 22:37:03,371] Trial 21 finished with value: 0.974296205630355 and parameters: {'n_estimators': 100, 'max_depth': 21, 'min_samples_leaf': 3}. Best is trial 20 with value: 0.9761613691931541.
[I 2022-06-29 22:37:04,841] Trial 22 finished with value: 0.9776004909481435 and parameters: {'n_estimators': 100, 'max_depth': 20, 'min_samples_leaf': 3}. Best is trial 22 with value: 0.9776004909481435.
[I 2022-06-29 22:37:06,350] Trial 23 finished with value: 0.9773145309625996 and parameters: {'n_estimators': 101, 'max_depth': 20, 'min_samples_leaf': 3}. Best is trial 22 with value: 0.9776004909481435.
[I 2022-06-29 22:37:07,977] Trial 24 finished with value: 0.9776416539050536 and parameters: {'n_estimators': 110, 'max_depth': 19, 'min_samples_leaf': 3}. Best is trial 24 with val

CPU times: user 55.8 s, sys: 129 ms, total: 55.9 s
Wall time: 55.8 s


In [39]:
# Опять рассчитаем f1_score для тестовой выборки
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model.fit(X_train, y_train)
y_test_pred = model.predict(X_test)
print('f1_score на тестовом наборе: {:.2f}'.format(metrics.f1_score(y_test, y_test_pred)))

f1_score на тестовом наборе: 0.82


#### Метрика не улучшилась.